In [1]:
# Import dependencies
import os
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
# Define the data_load function to read and process the data for a single stock
def data_load(file_path):
    try:
        data = pd.read_csv(file_path, parse_dates=['Date'])
        return data
    except Exception as e:
        print(f"Error loading data from {file_path}: {e}")
        return None

# Set path to Data folder
data_path = os.path.join(os.getcwd(), 'Data')

In [3]:
# Create a list to store all predictions
stock_predictions = []

# Use a for loop to iterate data_load over each CSV in the Data folder in order to get individual results for each stock
for stock in os.listdir(data_path):
    if stock.endswith('.csv'):
        file_path = os.path.join(data_path, stock)
        data = data_load(file_path)

        if data is None:
            continue

        # Filter this stock's data to only use entries from January 1st, 2020 onward
        start_date = datetime(2020, 1, 1)
        filtered_data = data[data['Date'] >= start_date]

        # Define new features in the data for the previous day's close and previous day's volume. Then use 'dropna()' to drop non-values.
        filtered_data.loc[:, 'Prev_Day_Close'] = filtered_data['Close'].shift(1)
        filtered_data.loc[:, 'Prev_Day_Vol'] = filtered_data['Volume'].shift(1)
        filtered_data = filtered_data.dropna()

        # Ensure data is sorted by date
        filtered_data = filtered_data.sort_values(by='Date')

        # Define features (X) and target (y) for training purposes
        X = filtered_data[['Prev_Day_Close', 'Prev_Day_Vol']]
        y = filtered_data['Close']

        # Split data into test and training sets chronologically
        cutoff_date = datetime(2023, 12, 31)
        X_train = X[filtered_data['Date'] <= cutoff_date]
        y_train = y[filtered_data['Date'] <= cutoff_date]
        X_test = X[filtered_data['Date'] > cutoff_date]
        y_test = y[filtered_data['Date'] > cutoff_date]

        if not X_train.empty and not y_train.empty:
            # Train stock_model
            stock_model = RandomForestRegressor(n_estimators=100, random_state=42)
            stock_model.fit(X_train, y_train)

            # Evaluate the stock_model using mse (mean squared error)
            y_pred = stock_model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            print(f'{stock}: Mean Squared Error: {mse}')
        else:
            print(f"No data points available for training for stock: {stock}")

        # Make remainder of the year predictions for this stock
        if not filtered_data.empty:
            future_dates = pd.date_range(start='2024-07-01', end='2024-12-31', freq='B')
            future_data = pd.DataFrame({
                'Date': future_dates,
                'Prev_Day_Close': filtered_data['Close'].iloc[-1],  
                'Prev_Day_Vol': filtered_data['Volume'].iloc[-1]  
            })
            stock_projections = stock_model.predict(future_data[['Prev_Day_Close', 'Prev_Day_Vol']])
            future_data['Projected_Close'] = stock_projections
            future_data['Stock'] = stock.replace('.csv', '')  
            stock_predictions.append(future_data)
        else:
            print(f"No data available for future predictions for stock: {stock}")

# Join all predictions into one DataFrame using concat
stock_predictions_df = pd.concat(stock_predictions, ignore_index=True)

print(stock_predictions_df.head())

C:\Users\danba\AppData\Local\Temp\ipykernel_30344\3272540238.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc[:, 'Prev_Day_Close'] = filtered_data['Close'].shift(1)
C:\Users\danba\AppData\Local\Temp\ipykernel_30344\3272540238.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc[:, 'Prev_Day_Vol'] = filtered_data['Volume'].shift(1)


AAPL.csv: Mean Squared Error: 7.545793172651929


C:\Users\danba\AppData\Local\Temp\ipykernel_30344\3272540238.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc[:, 'Prev_Day_Close'] = filtered_data['Close'].shift(1)
C:\Users\danba\AppData\Local\Temp\ipykernel_30344\3272540238.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc[:, 'Prev_Day_Vol'] = filtered_data['Volume'].shift(1)


ADBE.csv: Mean Squared Error: 237.13527766031015


C:\Users\danba\AppData\Local\Temp\ipykernel_30344\3272540238.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc[:, 'Prev_Day_Close'] = filtered_data['Close'].shift(1)
C:\Users\danba\AppData\Local\Temp\ipykernel_30344\3272540238.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc[:, 'Prev_Day_Vol'] = filtered_data['Volume'].shift(1)


...

VRTX.csv: Mean Squared Error: 268.1956319187371


C:\Users\danba\AppData\Local\Temp\ipykernel_30344\3272540238.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc[:, 'Prev_Day_Close'] = filtered_data['Close'].shift(1)
C:\Users\danba\AppData\Local\Temp\ipykernel_30344\3272540238.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc[:, 'Prev_Day_Vol'] = filtered_data['Volume'].shift(1)


WBA.csv: Mean Squared Error: 1.3184863057124643


C:\Users\danba\AppData\Local\Temp\ipykernel_30344\3272540238.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc[:, 'Prev_Day_Close'] = filtered_data['Close'].shift(1)
C:\Users\danba\AppData\Local\Temp\ipykernel_30344\3272540238.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc[:, 'Prev_Day_Vol'] = filtered_data['Volume'].shift(1)


WDC.csv: Mean Squared Error: 3.177212870049628
        Date  Prev_Day_Close  Prev_Day_Vol  Projected_Close Stock
0 2024-07-01      166.899994      48917700       166.673201  AAPL
1 2024-07-02      166.899994      48917700       166.673201  AAPL
2 2024-07-03      166.899994      48917700       166.673201  AAPL
3 2024-07-04      166.899994      48917700       166.673201  AAPL
4 2024-07-05      166.899994      48917700       166.673201  AAPL
